# Lizard cipher function (state update and keystream)

In [3]:
def keystream(B,S):
    '''
    Keystream bit function
    '''
    L = B[7] + B[11] + B[30] + B[40] + B[45] + B[54] + B[71]
    Q = B[4]*B[21] + B[9]*B[52] + B[18]*B[37] + B[44]*B[76]
    T = B[5] + B[8]*B[82] + B[34]*B[67]*B[73] + B[2]*B[28]*B[41]*B[65] + B[13]*B[29]*B[50]*B[64]*B[75] \
    + B[6]*B[14]*B[26]*B[32]*B[47]*B[61] + B[1]*B[19]*B[27]*B[43]*B[57]*B[66]*B[78]
    T_c = S[23] + S[3]*S[16] + S[9]*S[13]*B[48] + S[1]*S[24]*B[38]*B[63]
    
    out = L + Q + T + T_c + b0 + b0
    return out

def NFSR1_bk_update(S):
    '''
    NFSR1 update function in backward direction
    '''
    temp = S[30]  + S[1]  +  S[4]  + S[5]  +  S[14] + S[16] + S[17] + S[19] +  S[24] + S[7]*S[17] + S[7]*S[19] + S[11]*S[20] \
        + S[13]*S[18] + S[16]*S[20] + S[19]*S[21] + S[3]*S[11]*S[21] + S[3]*S[18]*S[21] + S[6]*S[19]*S[20] \
        + S[7]*S[17]*S[21] + S[7]*S[19]*S[21] + S[11]*S[18]*S[21] + S[19]*S[20]*S[21] + S[3]*S[6]*S[11]*S[20]\
        + S[3]*S[6]*S[18]*S[20] + S[3]*S[11]*S[20]*S[21] + S[3]*S[18]*S[20]*S[21] + S[6]*S[7]*S[17]*S[20] \
        + S[6]*S[7]*S[19]*S[20] + S[6]*S[11]*S[18]*S[20] + S[7]*S[17]*S[20]*S[21] + S[7]*S[19]*S[20]*S[21]\
        + S[11]*S[18]*S[20]*S[21] + b0 + b0
    
    S = [temp] + S[:30]
    return S

def NFSR2_bk_update(B,s0):
    '''
    NFSR2 update function in backward direction
    '''
    temp = s0 + B[89] + B[23] + B[48] + B[78] + B[83] + B[2]*B[58] + B[9]*B[11] + B[14]*B[15] + B[24]*B[52] \
    + B[34]*B[41] + B[54]*B[57] + B[59]*B[73] + B[19]*B[21]*B[22] + B[61]*B[67]*B[71] \
    + B[76]*B[79]*B[80]*B[82] + b0 + b0
    
    B = [temp] + B[:89]
    return B
    
def NFSR1_fw_update(S):
    '''
    NFSR1 update function in forward direction
    '''
    temp = S[0]  + S[2]  +  S[5]  + S[6]  +  S[15] + S[17] + S[18] + S[20] +  S[25] + S[8]*S[18] + S[8]*S[20] + S[12]*S[21] \
        + S[14]*S[19] + S[17]*S[21] + S[20]*S[22] + S[4]*S[12]*S[22] + S[4]*S[19]*S[22] + S[7]*S[20]*S[21] \
        + S[8]*S[18]*S[22] + S[8]*S[20]*S[22] + S[12]*S[19]*S[22] + S[20]*S[21]*S[22] + S[4]*S[7]*S[12]*S[21]\
        + S[4]*S[7]*S[19]*S[21] + S[4]*S[12]*S[21]*S[22] + S[4]*S[19]*S[21]*S[22] + S[7]*S[8]*S[18]*S[21] \
        + S[7]*S[8]*S[20]*S[21] + S[7]*S[12]*S[19]*S[21] + S[8]*S[18]*S[21]*S[22] + S[8]*S[20]*S[21]*S[22]\
        + S[12]*S[19]*S[21]*S[22] + b0 + b0
    
    for i in range(30):
        S[i]=S[i + 1]
    S[30]=temp
    return(S)

    
def NFSR2_fw_update(B,s0):
    '''
    NFSR2 update function in forward direction
    '''
    temp = s0 + B[0] + B[24] + B[49] + B[79] + B[84] + B[3]*B[59] + B[10]*B[12] + B[15]*B[16] + B[25]*B[53] \
    + B[35]*B[42] + B[55]*B[58] + B[60]*B[74] + B[20]*B[22]*B[23] + B[62]*B[68]*B[72] \
    + B[77]*B[80]*B[81]*B[83] + b0 + b0
    
    for i in range(89):
        B[i]=B[i + 1]
    B[89]=temp
    return(B)


def intersection(A,B):
    '''
    It returns the common element in the array A and B
    '''
    temp = []
    for i in range(len(A)):
        if A[i] in B:
            temp.append(A[i])
    return temp


# Time-Memory-Data-Trade-off attack algorithm

In [15]:
import time
import csv 
import numpy as np
import gurobipy as gp
from gurobipy import GRB

start_1 = time.time()

#Output is written into both .txt and .csv files

filename = 'Lizard_TMDTO_bkd.txt'
csv_file = 'Lizard_TMDTO_bkd.csv'

n_keys = 15 #Number of keystream bits required


keys_b = 0   #Number of backward keystream bits 
keys_f = n_keys-keys_b   #Number of forward keystream bits


#Pattern in which the keystream bit equations needed to be solved
pattern = [-i for i in range(1,keys_b+1,1)] + [i for i in range(keys_f)]

#pattern = [0]
#for i in range(1,max(keys_f,keys_b)+1,1):
#    if i < keys_b+1:
#        pattern = pattern + [-i]
#    if i < keys_f:
#        pattern = pattern + [i]

#pattern = [i for i in range(-keys_b,0,1)] + [i for i in range(keys_f)]

print("Pattern of the keystream bit equations ", pattern)
state_len = 121   #Length of the internal state bits of the cipher

#Length of the registers R1 and R2
R1_len = 90
R2_len = 31


#Defining the variables for internal state of the cipher
V = BooleanPolynomialRing(state_len,['b%d'%(i) for i in range(R1_len)] + ['s%d'%(i) for i in range(R2_len)])
V.inject_variables()
g = list(V.gens()) 

#Assigning variables to registers: NFSR B and NFSR S
B = g[:R1_len] 
S = g[R1_len:]

R = []  # For Recovery variables
F = []  # For Fixed variables
G = []  # For Guess variables

#In case, if we fix some variables to zero beforehand
for i in F:
    if i < R1_len:
        B[i] = 0
    else:
        S[i-R1_len] = 0
        

linear_b = [0]*keys_b  #It will contain the linear variables for backward keystream equations
linear_f = [0]*keys_f  #It will contain the linear variables for forward keystream equations
n_mon = 0              #Variable for no. of monomials in a given equation
Z_f = [0]*keys_f       #For forward keystream bit equations
Z_b = [0]*keys_b       #For backward keystream bit equations


#Generate and store the data corresponding to backward keystream bits
for loop in range(keys_b):
    #Updating both registers S and B
    S = NFSR1_bk_update(S)                  
    B = NFSR2_bk_update(B,S[0])

    Z_b[loop] = keystream(B,S)            #Storing the keystream bit equation
    monom = list(Z_b[loop].monomials())       #Storing the monomials of the given equation
    linear_b[loop] = [m for m in monom if m.degree() == 1]       #Storing the monomials of degree 1
    
    n_mon += (len(monom) - len(linear_b[loop]))     #No. of non-linear monomials of degree > 1
    
    #print("backward loop 1st part ", loop)

#Initialise the register again with the state variables
B = g[:R1_len]  
S = g[R1_len:]


#Fix the variable to 0 which are in Fixed category
for i in F:
    if i < R1_len:
        B[i] = 0
    else:
        S[i-R1_len] = 0

#Generate and store the data corresponding to backward keystream bits 
for loop in range(keys_f):
    Z_f[loop] = keystream(B,S)                  #Storing the keystream bit equation
    monom = list(Z_f[loop].monomials())         #Storing the monomials of the given equation
    linear_f[loop] = [m for m in monom if m.degree() == 1]    #Storing the monomials of degree 1
    
    n_mon += (len(monom) - len(linear_f[loop]))       #Counting the number of monomials of degree > 1

    #print("forward loop 1st part ",loop)
    
    if loop != keys_f-1:
        #updating both registers S and B
        B=NFSR2_fw_update(B,S[0])
        S=NFSR1_fw_update(S) 
#print("1st part done")



index = 0
Z = [0]*(keys_f+keys_b)          # To store the keystream bit equations as per the pattern
linear = [0]*(keys_f + keys_b)   # Storing the corresponding linear terms
linear_new = [0]*(keys_f + keys_b)  # Storing the linear terms which did not occured in the linear terms of the previous equations


for i in pattern: 
    if i < 0:
        Z[index] = Z_b[-i-1]
        linear[index] = linear_b[-i-1]
    else:
        Z[index] = Z_f[i]
        linear[index] = linear_f[i]
    index += 1

    
linear_all = []                  # Storing all the linear terms of all keystream equations
for loop in range(len(Z)):
    lin = []
    for i in linear[loop]:
        if g.index(i) not in linear_all:
            lin = lin + [g.index(i)]     #storing the new linear terms which were not in the linear terms of previous equations

    linear_new[loop] = lin
    linear_all = linear_all + lin

#########Gurobi Modelling Starts###############
model = gp.Model()


v = [0,1,2]             # Temporary variable to be used for gurobi variable definition
aux = [i for i in range(n_mon)]

#Let R, F, G represents the class of Recovery, Fixed bit, and Guess bit
#Since these are set and we need variables to denote R, F and G therefore let 2=R, 3=F and 4=G
#x represents variables for each register and m contains 3 variables for each x[i] i.e. for R, F and G
x = model.addVars(len(B) + len(S), lb = 2, ub = 4, vtype = GRB.INTEGER, name = "x")
m = model.addVars(len(B) + len(S), len(v), vtype = GRB.BINARY, name = "m")
q = model.addVars(len(aux), vtype = GRB.BINARY, name = "q")  #Auxiliary variables for each non-linear term
#p = model.addVars(len(aux), vtype = GRB.BINARY, name = "p")
#r = model.addVars(len(aux), vtype = GRB.BINARY, name = "r")
model.update()

model.addConstrs((m[i,0] == 1) for i in R)
model.addConstrs((m[i,1] == 1) for i in F)
model.addConstrs((m[i,2] == 1) for i in G)

#Objective function is to maximize recovery bit and minimize the number of FIXED bit
model.setObjectiveN(gp.quicksum((m[i,0]) for i in range(state_len)),0,1, GRB.MAXIMIZE)
model.setObjectiveN(gp.quicksum((m[i,1]) for i in range(state_len)),1,0, GRB.MINIMIZE)


#Constraint to denote that each variable x[i] belongs to exactly one category out of R, F and G category
model.addConstrs((x[i] == (2*m[i,0] + 3*m[i,1] + 4*m[i,2])) for i in range(state_len))
model.addConstrs(m[i,0] + m[i,1] + m[i,2] == 1 for i in range(state_len))


#Number of recovered bit should be less than or equal to the number of keystream equations
#model.addConstr(gp.quicksum(m[i,0] for i in range(state_len)) <= n_keys)

#Constraint: All variables which do not come under the linear terms cannot be in recovery category
model.addConstr(gp.quicksum(m[i,0] for i in range(state_len) if i not in linear_all) == 0)


#Analysing each keystream bit equation
count = 0
cnt = 0
for loop in range(keys_f+keys_b):
    z = Z[loop]
    z_monom = np.array(z.monomials())  #Monomials of the equation z

    z_linear = [g.index(i) for i in z_monom if i.degree() == 1]   #Linear terms of the equation z


    #Constraint: one of the linear term variables which is not in the previous linear term has at most one recovery bit
    model.addConstr(gp.quicksum(m[i,0] for i in linear_new[loop]) <= 1)    

    
    #Analyzing each monomial (with degree > 1) of the keystream equation
    for mon in z_monom:
        if mon.degree() > 1:
            var = list(mon.variables())
            var =  [g.index(i) for i in var]      #Containing the index of variable in the monomial, mon
            
            inters = [0]*len(linear_new)   
            n = [0]*len(linear_new)
            for l in range(len(linear_new)):
                inters[l] = intersection(var,linear_new[l])   #Intersection of var with linear_new[l]
                n[l] = len(inters[l])         #Length of inters[l]

            C_prev = [n[i] for i in range(loop)]  #No. of variables in var which are present as a linear terms in the previous equations
            C_current = [n[loop]]         #No. of variables in var which are present in the linear term of z
            C_future = [n[i] for i in range(loop+1,n_keys,1)]   #No. of variables in var which are present in linear terms of the next equations


            #Case: current or future linear terms present as a variable in mon
            if (sum(C_current + C_future) > 0):
                #if current or future recovery bit found then fix a variable in mon to zero
                prev = []
                for k in range(loop):
                    prev = prev + inters[k]

                var_cur_fut = []
                for k in range(loop,len(linear_new),1):
                    var_cur_fut = var_cur_fut + inters[k]

                #Constraint: If any of the variables in var comes in recovery category and is in the linear_new of current or upcoming equations, then make the monommial to 0 by fixing some variable to 0 
                model.addConstr((q[count] == 0) >> (gp.quicksum(m[i,0] for i in var_cur_fut) == 0))
                model.addConstr((q[count] == 1) >> (gp.quicksum(m[i,1] for i in var) >= 1))
                count += 1

            #if (sum(C_prev + C_current + C_future) == 0): 
            #    model.addConstr(gp.quicksum((m[i,0]) for i in var) == 0)
    #print("2nd part loop ", loop)
#print("2nd part done")

model.params.outputflag = 0 #To display only the required information
start_2 = time.time()
model.optimize()        #Solve the above MILP model
time_2 = time.time() - start_2


print("Keystream = ",n_keys)
print("status = ", model.STATUS)


#Writing the information in a text file
f = open(filename,'a')
f.write("\nKeystream = " + str(n_keys))
f.write("\nForward = " + str(keys_f) + "; Backward = " + str(keys_b))
f.write("\nStatus " + str(model.STATUS) + "\n")
f.write("\nPattern = " + str(pattern) + "\n")

if model.STATUS == 2:  #If an optimal solution is available
    val = model.x     #Extract the solution
    R = []
    F = []
    G = []
    
    #Identifying the output and dividing as per the category
    for i in range(state_len):
        if round(val[i]) == 2:
            R.append(i)
        if round(val[i]) == 3:
            F.append(i)
        if round(val[i]) == 4:
            G.append(i)


    #Storing the variable as per the indices in R,F and G category
    Rec = [g[i] for i in R]
    Fix = [g[i] for i in F]
    Guess = [g[i] for i in G]
    
    print("Recovery Bit ",Rec)
    print("Fixed Bit ",Fix)
    #print("Guessed Bit ",Guess)

    f.write(str("Recovery Bit ") + str(Rec) + "\n")
    f.write(str("Fixed Bit ") + str(Fix) + "\n")
    f.write(str("Guessed Bit ") + str(Guess) + "\n")


    print("\nNumber of recovery bit = ", len(R),"\nNumber of fixed bit = ", len(F), "\n")
    
    #verification of the obtained values
    if (len(R) + len(F) + len(G) == state_len) \
    and len(intersection(R,F) + intersection(F,G) + intersection(R,G)) == 0:
        print("Length verified\n")
        f.write("\nLength Verified\n\n")
        
time_1 = time.time()-start_1
print("Gurobi Time = ", time_2)
print("Overall Time = ", time_1, '\n')

f.write("\n\nGurobi Time = " + str(time_2))
f.write("\nOverall Time = " + str(time_1))
f.close()





Pattern of the keystream bit equations  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Defining b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13, b14, b15, b16, b17, b18, b19, b20, b21, b22, b23, b24, b25, b26, b27, b28, b29, b30, b31, b32, b33, b34, b35, b36, b37, b38, b39, b40, b41, b42, b43, b44, b45, b46, b47, b48, b49, b50, b51, b52, b53, b54, b55, b56, b57, b58, b59, b60, b61, b62, b63, b64, b65, b66, b67, b68, b69, b70, b71, b72, b73, b74, b75, b76, b77, b78, b79, b80, b81, b82, b83, b84, b85, b86, b87, b88, b89, s0, s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15, s16, s17, s18, s19, s20, s21, s22, s23, s24, s25, s26, s27, s28, s29, s30
Keystream =  15
status =  2
Recovery Bit  [b36, b39, b46, b54, b56, b57, b59, b61, b62, b64, b65, b66, b67, b68, s27]
Fixed Bit  [b20, b29, b30, b31, b32, b33, b40, b41, b43, b72, b73, b74, b75, s2, s28]

Number of recovery bit =  15 
Number of fixed bit =  15 

Length verified

Gurobi Time =  0.7114388942718506
Ov

# Verification Process

In [16]:
############Verification whether the obtained variables are correct########################
print("\n############Verification Starts#############")
R = R
Rec = [g[i] for i in R]
print("Recovered Bits are ", Rec)
Fix = [g[i] for i in F]
print("Fixed Bits are ", Fix)

#Assigning variables to the registers
B = g[:90] 
S = g[90:]

#Fixing the variables in F category to 0
for i in F:
    if i < 90:
        B[i] = 0
    else:
        S[i-90] = 0

        
Z_f = [0]*keys_f
Z_b = [0]*keys_b

for loop in range(keys_b):
    S = NFSR1_bk_update(S)
    B = NFSR2_bk_update(B,S[0])

    Z_b[loop] = keystream(B,S)   #Generating keystream equations in backward directions


B = g[:90] 
S = g[90:]

for i in F:
    if i < 90:
        B[i] = 0
    else:
        S[i-90] = 0


for loop in range(keys_f):
    Z_f[loop] = keystream(B,S)     #Generating keystream equations in forward directions

    if loop != keys_f-1:
        B=NFSR2_fw_update(B,S[0])
        S=NFSR1_fw_update(S) 


index = 0
Z = [0]*(keys_f+keys_b)   

#Storing all equations in Z, ordered as per the pattern
for i in pattern:
    if i < 0:
        Z[index] = Z_b[-i-1]
    else:
        Z[index] = Z_f[i]
    index += 1


R_past = []  #Defined for the recovery variables from the previous equations
L_rec = []   #Defined for the recovery variables which are in the linear terms of the equations
#cnt = 0
flag = 1
for i in range(keys_f+keys_b):
    z = Z[i]
    monom = list(z.monomials())   #Monomials of the equation z
    t_var = list(z.variables())   #Storing all the variables in z

    R_past = R_past + list(set(L_rec))   #Storing the recovery variables of the previous equations
    m_rec = []                   
    print("\n", i, end = " = ")
    for m in monom:
        m_var = list(m.variables())
        if (len(m_var) == 1) and (len(intersection(m_var,Rec)) > 0):
            L_rec = L_rec + m_var             #Storing the recovery variables
            print(intersection(m_var,Rec),end=" + ")

        if (len(m_var) > 1) and (len(intersection(m_var,Rec)) > 0):
            m_rec = m_rec + [m]               #Storing the monomials (deg > 1) and contains recovery variables
            print(intersection(m_var,Rec),"*G + ",end=" ")
    #print("monomials containing recovery bits (degree > 1) ",m_rec)

    for m in m_rec:
        m_var = list(m.variables())
        #Now L_rec contains current and past recovery bit in linear terms
        #Whereas R_past contains only the past recovery bit in linear terms
        
        #Check if the monomials contains any current or future recovery variables
        if len(intersection(m_var,Rec)) > len(intersection(m_var,R_past)):
            f = open(filename,'a')
            f.write("\nError --- Future recovery bit found\n" + str(m))
            f.close()
            print("Error --- Future recovery bit found")
            print(m)
            flag = 0   #It implies that the verification failed

    L_rec = list(set(L_rec))
    #if len(L_rec) != cnt + 1:
    #    flag = 0
    #cnt = len(L_rec)
    #print("Recovery bits till now in linear terms ",L_rec)


f = open(filename,'a')
if flag == 0:
    print("\n\n###################Verification Process Failed##################\n")
    f.write("\n\n###################Verification Process Failed##################\n")
else:
    print("\n\n####################Verification Process Passed######################\n")
    f.write("\n\n####################Verification Process Passed######################\n")
    with open(csv_file,'a') as f:
        writer = csv.writer(f)
        #writer.writerow(['Number of Recovered bits','Number of Fixed bits','Recovered bits','Fixed bits'])
        writer.writerow([keys_f,keys_b,len(R),len(F),Rec,Fix,pattern])
f.close()



############Verification Starts#############
Recovered Bits are  [b36, b39, b46, b54, b56, b57, b59, b61, b62, b64, b65, b66, b67, b68, s27]
Fixed Bits are  [b20, b29, b30, b31, b32, b33, b40, b41, b43, b72, b73, b74, b75, s2, s28]

 0 = [b54] + 
 1 = [b46] + 
 2 = [b54] *G +  [b46] *G +  [b56] + 
 3 = [b57] + 
 4 = [b56] *G +  [s27] + 
 5 = [b57] *G +  [b59] + 
 6 = [b56] *G +  [b36] + [b46] + [b54] *G +  
 7 = [b59] *G +  [b61] + 
 8 = [b59] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b46] *G +  [b56] *G +  [b62] + 
 9 = [b54] *G +  [b56, b59] *G +  [b61] *G +  [b46] *G +  [b39] + [b54] + [b57] *G +  
 10 = [b61] *G +  [b36, b57] *G +  [b57] *G +  [b62] *G +  [b62] *G +  [b54] *G +  [b64] + [s27] + 
 11 = [b62] *G +  [b39] *G +  [b56] *G +  [b61] *G +  [b56] + [b59] *G + 